In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_inference_server.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/__init__.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/templates.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/base_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/relay.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/__init__.py
/kaggle/input/cmi-detect-behav

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def custom_pad_sequences(sequences, maxlen, dtype='float32'):
    """
    Pad or truncate sequences to maxlen using the median of the last 5 elements for padding.
    For sequences shorter than maxlen, compute the median of the last 5 elements (or all if < 5)
    and use it to pad to maxlen. For sequences longer than maxlen, truncate from the beginning
    (post-truncation).

    Args:
        sequences: List of arrays, each with shape (seq_len, num_features)
        maxlen: Target length for all sequences
        dtype: Data type for the output array

    Returns:
        Padded array with shape (num_samples, maxlen, num_features)
    """
    num_samples = len(sequences)
    num_features = sequences[0].shape[1]
    padded = np.zeros((num_samples, maxlen, num_features), dtype=dtype)

    for i, seq in enumerate(sequences):
        seq_len = len(seq)
        if seq_len >= maxlen:
            # Truncate from the beginning (post-truncation)
            padded[i] = seq[-maxlen:]  # Take the last maxlen elements
        else:
            # Compute median of last 5 elements (or all if seq_len < 5)
            last_n = seq[-min(3, seq_len):]  # Take last 5 elements or all if fewer
            median_values = np.median(last_n, axis=0)  # Median across time axis
            # Calculate how many padding elements are needed
            pad_length = maxlen - seq_len
            # Create padding array by repeating median values
            padding = np.repeat(median_values[np.newaxis, :], pad_length, axis=0)
            # Concatenate original sequence with padding
            padded[i] = np.concatenate([seq, padding], axis=0)
    
    return padded

def remove_gravity_from_acc(acc_data, rot_data):
    """
    Remove gravity component from accelerometer data using quaternion rotation data.

    Args:
        acc_data: DataFrame with columns ['acc_x', 'acc_y', 'acc_z']
        rot_data: DataFrame with columns ['rot_x', 'rot_y', 'rot_z', 'rot_w']

    Returns:
        Numpy array of linear acceleration with gravity removed
    """
    acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
   
    for i in range(len(acc_values)):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
            linear_accel[i, :] = acc_values[i, :]
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    """
    Calculate angular velocity from quaternion data.

    Args:
        rot_data: DataFrame with columns ['rot_x', 'rot_y', 'rot_z', 'rot_w']
        time_delta: Time interval between samples (default: 1/200 seconds)

    Returns:
        Numpy array of angular velocities
    """
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_vel = np.zeros((len(quat_values), 3))
   
    for i in range(len(quat_values) - 1):
        q_t, q_t_plus_dt = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q_t)) or np.all(np.isnan(q_t_plus_dt)):
            continue
        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            pass
    return angular_vel

def calculate_angular_distance(rot_data):
    """
    Calculate angular distance between consecutive quaternions.

    Args:
        rot_data: DataFrame with columns ['rot_x', 'rot_y', 'rot_z', 'rot_w']

    Returns:
        Numpy array of angular distances
    """
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_dist = np.zeros(len(quat_values))
   
    for i in range(len(quat_values) - 1):
        q1, q2 = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q1)) or np.all(np.isnan(q2)):
            continue
        try:
            r1, r2 = R.from_quat(q1), R.from_quat(q2)
            relative_rotation = r1.inv() * r2
            angular_dist[i] = np.linalg.norm(relative_rotation.as_rotvec())
        except ValueError:
            pass
    return angular_dist